In [6]:
import os
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.agent_toolkits.sql.base import create_sql_agent
from langchain_community.llms import Cohere
from langchain_community.llms import Together


In [7]:
os.environ['COHERE_API_KEY'] ="il5TNhrf0DPw17bURNZPOJdNs4XnVEnhwT8glPzX"
together_api_key = os.getenv("TOGETHER_API_KEY", "6cd0fed4610e7c1919a1d450471b655516ef38eb68704967b264b46d97cd7b26")

In [21]:
db_user = "root"
db_password = ""
db_host = "localhost"
db_name = "classicmodels"
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")


# db_user = "hstoa"
# db_password = "password"
# db_host = "196.188.63.235"
# db_name = "hstoa_testdb"
# db = SQLDatabase.from_uri(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_name}")

# db_user = "root"
# db_password ="Password.01"
# db_host = "196.188.63.95"
# db_name = "tenant_55"

# db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")


In [22]:
# Initialize Cohere LLM
llm = Cohere(model="command-xlarge", temperature=0)

# Set up toolkit and agent
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

In [ ]:
# # Initialize Together AI LLM (Mistral 7B Instruct)
# llm = Together(
#     model="mistralai/Mistral-7B-Instruct-v0.2",
#     temperature=0,
#     max_tokens=512,
#     together_api_key=together_api_key
# )

C:\Users\zelalem.wubet\AppData\Local\Temp\ipykernel_25344\3880311971.py:2: LangChainDeprecationWarning: The class `Together` was deprecated in LangChain 0.0.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-together package and should be used instead. To use it run `pip install -U :class:`~langchain-together` and import as `from :class:`~langchain_together import Together``.
  llm = Together(


In [13]:
# # Change the model name to a serverless one
# llm = Together(
#     model="togethercomputer/llama-2-13b-chat",  # Smaller model that works serverless
#     temperature=0.2,
#     max_tokens=512,
#     together_api_key=together_api_key
# )

In [ ]:
# from langchain.agents.agent_types import AgentType
# # 3. Create the SQL Agent using LangChain
# agent_executor = create_sql_agent(
#     llm=llm,
#     toolkit=SQLDatabaseToolkit(db=db, llm=llm),
#     agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
#     verbose=True,
#     handle_parsing_errors=True,
# )

In [13]:
agent_executor.run("Describe the Order related table and how they are related")


C:\Users\zelalem.wubet\AppData\Local\Temp\ipykernel_25344\3277361352.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent_executor.run("Describe the Order related table and how they are related")




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:customers, employees, offices, orderdetails, orders, payments, productlines, products I should look at the order related tables.
Action: sql_db_schema
Action Input: orders, orderdetails
CREATE TABLE orderdetails (
	`orderNumber` INTEGER(11) NOT NULL, 
	`productCode` VARCHAR(15) NOT NULL, 
	`quantityOrdered` INTEGER(11) NOT NULL, 
	`priceEach` DECIMAL(10, 2) NOT NULL, 
	`orderLineNumber` SMALLINT(6) NOT NULL, 
	PRIMARY KEY (`orderNumber`, `productCode`), 
	CONSTRAINT orderdetails_ibfk_1 FOREIGN KEY(`orderNumber`) REFERENCES orders (`orderNumber`), 
	CONSTRAINT orderdetails_ibfk_2 FOREIGN KEY(`productCode`) REFERENCES products (`productCode`)
)COLLATE utf8mb4_general_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from orderdetails table:
orderNumber	productCode	quantityOrdered	priceEach	orderLineNumber
10100	S18_1749	30	136.00	3
10100	S18_2248	50	55.09	2
10100	S18_4409	22	75.46	4
*/


CREATE TABLE o

'The Order table contains information about the order itself, such as the order number, order date, required date, shipped date, status, comments, and customer number.  The OrderDetails table contains information about the details of each order, such as the product code, quantity ordered, price each, and order line number.  The OrderDetails table has a foreign key constraint on the order number column that references the order number column in the Orders table, indicating that each row in the OrderDetails table is associated with a specific row in the Orders table.'

In [16]:
agent_executor.run("Describe the PurchaseDetails table")



> Entering new SQL Agent Executor chain...
 Action: sql_db_list_tables
Action Input: 
customers, employees, offices, orderdetails, orders, payments, productlines, productsThe PurchaseDetails table is related to purchases and product details. We should understand the products table to describe PurchaseDetails.

Thought: I will try to query the schema of the products table
Action: sql_db_schema
Action Input: products

CREATE TABLE products (
	`productCode` VARCHAR(15) NOT NULL, 
	`productName` VARCHAR(70) NOT NULL, 
	`productLine` VARCHAR(50) NOT NULL, 
	`productScale` VARCHAR(10) NOT NULL, 
	`productVendor` VARCHAR(50) NOT NULL, 
	`productDescription` TEXT NOT NULL, 
	`quantityInStock` SMALLINT(6) NOT NULL, 
	`buyPrice` DECIMAL(10, 2) NOT NULL, 
	`MSRP` DECIMAL(10, 2) NOT NULL, 
	PRIMARY KEY (`productCode`), 
	CONSTRAINT products_ibfk_1 FOREIGN KEY(`productLine`) REFERENCES productlines (`productLine`)
)ENGINE=InnoDB COLLATE utf8mb4_general_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from p

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Parsing LLM output produced both a final answer and a parse-able action:: Thought: I now know the final answer.
Final Answer: The PurchaseDetails table contains information on products, including product name, buy price, vendor, and description. It appears to be related to the orders table, which would contain information on purchases. 
Thought: I should try to query the schema of the orders table to understand PurchaseDetails further. 

Action: sql_db_schema
Action Input: orders
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [14]:
agent_executor.invoke({"input": "Find the top 5 products with the highest total sales revenue"}, config={"verbose": True})




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:customers, employees, offices, orderdetails, orders, payments, productlines, products I should look at the productlines and products tables to find the relevant columns for sales revenue.
Action: sql_db_schema
Action Input: productlines, products
CREATE TABLE productlines (
	`productLine` VARCHAR(50) NOT NULL, 
	`textDescription` VARCHAR(4000), 
	`htmlDescription` MEDIUMTEXT, 
	image MEDIUMBLOB, 
	PRIMARY KEY (`productLine`)
)COLLATE utf8mb4_general_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from productlines table:
productLine	textDescription	htmlDescription	image
Classic Cars	Attention car enthusiasts: Make your wildest car ownership dreams come true. Whether you are looking	None	None
Motorcycles	Our motorcycles are state of the art replicas of classic as well as contemporary motorcycle legends 	None	None
Planes	Unique, diecast airplane and helicopter replicas suitable for collections, as we

{'input': 'Find the top 5 products with the highest total sales revenue',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [ ]:
print(agent_executor.input_keys)


['input']


In [ ]:
agent_executor.run("List the top 3 countries with the highest number of orders")



> Entering new SQL Agent Executor chain...
 Action: sql_db_schema
Action Input: orders tableError: table_names {'orders table'} not found in databaseI should make sure I know the right name of the table before I try to query the schema.
 Action: sql_db_list_tables
Action Input: customers, employees, offices, orderdetails, orders, payments, productlines, productsI know the table is called `orders` so I should query the schema to see what columns it has.  Then I can decide which columns to use in my query.
 Action: sql_db_schema
Action Input: orders
CREATE TABLE orders (
	`orderNumber` INTEGER(11) NOT NULL, 
	`orderDate` DATE NOT NULL, 
	`requiredDate` DATE NOT NULL, 
	`shippedDate` DATE, 
	status VARCHAR(15) NOT NULL, 
	comments TEXT, 
	`customerNumber` INTEGER(11) NOT NULL, 
	PRIMARY KEY (`orderNumber`), 
	CONSTRAINT orders_ibfk_1 FOREIGN KEY(`customerNumber`) REFERENCES customers (`customerNumber`)
)ENGINE=InnoDB COLLATE utf8mb4_general_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from ord

"['France', 'USA', 'Australia']"